In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

This notebook was authored with assistance from [Ivan Nardini](https://github.com/inardini)

# Get started with Vertex AI Experiments

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/get_started_with_vertex_experiments.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fexperiments%2Fget_started_with_vertex_experiments.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/experiments/get_started_with_vertex_experiments.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/get_started_with_vertex_experiments.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview


This tutorial covers getting started with Vertex AI Experiments, and demonstrates how to use Vertex AI in production.

Learn more about [Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments), [Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata) and [Custom training](https://cloud.google.com/vertex-ai/docs/training/custom-training).

### Objective

In this tutorial, you learn how to use Vertex AI Experiments when training with Vertex AI.

This tutorial uses the following Google Cloud ML services:

- Vertex AI Experiments
- Vertex ML Metadata
- Vertex AI Training

The steps performed include:

- Local (notebook) training
    - Create an experiment.
    - Create a first run in the experiment.
    - Log parameters and metrics.
    - Create artifact lineage.
    - Visualize the experiment results.
    - Execute a second run.
    - Compare the two runs in the experiment.
- Cloud (Vertex AI) training
    - Within the training script:
        - Create an experiment.
        - Log parameters and metrics.
        - Create artifact lineage.
    - Create a Vertex AI training custom job.
    - Execute the custom job.
    - Visualize the experiment results,

### Recommendations

When doing E2E MLOps on Google Cloud, the following are some of the best practices for logging data when experimenting or formally training a model.

#### Python Logging

Use Python's logging package when doing ad-hoc training locally.

#### Cloud Logging

Use Cloud Logging when doing training on the cloud.

#### Experiments

Use Vertex AI Experiments in conjunction with logging when performing experiments to compare results for different experiment configurations.

### Dataset

This tutorial doesn't use a dataset. References to example datasets is for demonstration purposes only.

### Costs
This tutorial uses billable components of Google Cloud:

- Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [1]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

### Set Google Cloud project information and initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [4]:
PROJECT_ID = "projet-ia-448520" # "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [5]:
BUCKET_URI = "gs://cours1bucket" # f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [6]:
! gsutil mb -l $LOCATION $BUCKET_URI

Creating gs://cours1bucket/...
ServiceException: 409 A Cloud Storage bucket named 'cours1bucket' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


#### Service Account

**If you don't know your service account**, try to get your service account using `gcloud` command by executing the second cell below.

In [7]:
SERVICE_ACCOUNT = "837843744498-compute@developer.gserviceaccount.com" # "[your-service-account]"  # @param {type:"string"}

## Import Libraries

In [9]:
import os
import uuid

import google.cloud.aiplatform as aiplatform

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [10]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

#### Create experiment for tracking training related metadata

First, you create an experiment using the `init()` method and then initialize a run within the experiment using `start_run()`.

- `aiplatform.init()` - Create an experiment instance
- `aiplatform.start_run()` - Track a specific run within the experiment.

In [24]:
# Specify a name for the experiment
your_name = ...
EXPERIMENT_NAME = f"supaero-vertex-course-1-2_{your_name}"


In [18]:
# Create experiment
aiplatform.init(experiment=EXPERIMENT_NAME)
aiplatform.start_run("run-1")

Deleting Context : projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-course-1-1-run-1
Context deleted. . Resource name: projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-course-1-1-run-1
Deleting Context resource: projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-course-1-1-run-1
Delete Context backing LRO: projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-course-1-1-run-1/operations/2492575164902932480
Context resource projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-course-1-1-run-1 deleted.


ValueError: Experiment run run-1 already associated to tensorboard resource projects/837843744498/locations/us-central1/tensorboards/5595278334310219776/experiments/experiment-course-1-1/runs/run-1.
To delete backing tensorboard run, execute the following:
tensorboard_run_artifact = aiplatform.metadata.artifact.Artifact(artifact_name=f"experiment-course-1-1-run-1-tb-run")
tensorboard_run_resource = aiplatform.TensorboardRun(tensorboard_run_artifact.metadata["resourceName"])
tensorboard_run_resource.delete()
tensorboard_run_artifact.delete()

In [17]:
metaparams = {}
metaparams["units"] = 128
aiplatform.log_params(metaparams)

hyperparams = {}
hyperparams["epochs"] = 100
hyperparams["batch_size"] = 32
hyperparams["learning_rate"] = 0.01
aiplatform.log_params(hyperparams)

In [18]:

    aiplatform.end_run()

#### Log parameters for the experiment

Typically, an experiment is associated with a specific dataset and a model architecture. Within an experiment, you may have multiple training runs,
where each run involves a different configuration. For example:

- Data feeding, such as:
    - dataset split
    - dataset sampling and boosting
- Metaparameters, such as:
    - depth and width of layers
- Hyperparameters, such as:
    - batch size
    - learning rate

These configuration settings are referred to as parameters, which you store as key-value pairs using the method `log_params()`

In [25]:
import random

units_choice = [32, 64, 128, 256]
epochs_choice = [1, 10, 100, 1000]
batch_size_choice = [1, 8, 32, 64]
learning_rate_choice = [.0001, .01, .1, 1]

# Create experiment
aiplatform.init(experiment=EXPERIMENT_NAME)

for i in range(1, 11):
    aiplatform.start_run(f"run-{str(i)}")

    metaparams = {}
    metaparams["units"] = random.choice(units_choice)
    aiplatform.log_params(metaparams)

    hyperparams = {}
    hyperparams["epochs"] = random.choice(epochs_choice)
    hyperparams["batch_size"] = random.choice(batch_size_choice)
    hyperparams["learning_rate"] = random.choice(learning_rate_choice)
    aiplatform.log_params(hyperparams)

    metrics = {}
    metrics["test_acc"] = random.random() * 30 + 68
    metrics["train_acc"] = min(random.random() * 3 + metrics["test_acc"], 100)

    aiplatform.log_metrics(metrics)
    
    aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-1 to Experiment: supaero-vertex-course-1-1


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-2 to Experiment: supaero-vertex-course-1-1


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-3 to Experiment: supaero-vertex-course-1-1


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-4 to Experiment: supaero-vertex-course-1-1


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-5 to Experiment: supaero-vertex-course-1-1


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-6 to Experiment: supaero-vertex-course-1-1


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-7 to Experiment: supaero-vertex-course-1-1


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-8 to Experiment: supaero-vertex-course-1-1


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-9 to Experiment: supaero-vertex-course-1-1


Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/supaero-vertex-course-1-1-run-10 to Experiment: supaero-vertex-course-1-1


In [21]:
experiment_df

,experiment_name,run_name,run_type,state,param.batch_size,param.learning_rate,param.epochs,param.units,metric.train_acc,metric.test_acc
0,supaero-course-1-1,run-10,system.ExperimentRun,COMPLETE,8.0,0.0001,100.0,256.0,100.000000,95.712354
1,supaero-course-1-1,run-9,system.ExperimentRun,COMPLETE,64.0,0.1000,10.0,128.0,101.117122,99.558977
2,supaero-course-1-1,run-8,system.ExperimentRun,COMPLETE,64.0,0.0100,1.0,64.0,100.000000,84.759412
3,supaero-course-1-1,run-7,system.ExperimentRun,COMPLETE,1.0,1.0000,1.0,64.0,100.000000,84.887640
4,supaero-course-1-1,run-6,system.ExperimentRun,COMPLETE,64.0,0.0100,100.0,128.0,100.000000,91.225823
5,supaero-course-1-1,run-5,system.ExperimentRun,COMPLETE,64.0,1.0000,1.0,32.0,102.270010,99.515128
6,supaero-course-1-1,run-4,system.ExperimentRun,COMPLETE,1.0,0.1000,100.0,128.0,100.000000,89.742437
7,supaero-course-1-1,run-3,system.ExperimentRun,COMPLETE,8.0,1.0000,100.0,32.0,100.000000,95.709242
8,supaero-course-1-1,run-2,system.ExperimentRun,COMPLETE,64.0,0.1000,10.0,128.0,100.000000,91.013360
9,supaero-course-1-1,run-1,system.ExperimentRun,COMPLETE,32.0,0.0100,100.0,128.0,100.000000,93.166421


#### Log metrics for the experiment

At the completion or termination of a run within an experiment, you can log results that you use to compare runs. For example:

- evaluation metrics
- hyperparameter search selection
- time to train the model
- early stop trigger

These results are referred to as metrics, which you store as key-value pairs using the method `log_metrics()`

In [51]:
metrics = {}
metrics["test_acc"] = 98.7
metrics["train_acc"] = 99.3
aiplatform.log_metrics(metrics)

#### Get the experiment results

When you are finished with a run within an experiment, call `end_run()` method to complete the logging for that run.

Next, use the experiment name as a parameter to the method `get_experiment_df()` to get the results of the experiment as a pandas dataframe.

In [52]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)

                                            0
experiment_name      experiment-test-course-1
run_name                                run-1
run_type                 system.ExperimentRun
state                                COMPLETE
param.epochs                            100.0
param.batch_size                         32.0
param.units                             128.0
param.learning_rate                      0.01
metric.train_acc                         99.3
metric.test_acc                          98.7


#### Start subsequent run in an experiment

Next, create a second run for the same experiment. In this example, you change the metaparameter for `units` from 126 to 256, and log different metric results.

In [53]:
aiplatform.start_run("run-2")

metaparams = {}
metaparams["units"] = 256  # changed the value
aiplatform.log_params(metaparams)

hyperparams = {}
hyperparams["epochs"] = 100
hyperparams["batch_size"] = 32
hyperparams["learning_rate"] = 0.01
aiplatform.log_params(hyperparams)

metrics = {}
metrics["test_acc"] = 98.8  # value changed
metrics["train_acc"] = 99.5  # value changed
aiplatform.log_metrics(metrics)

Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-test-course-1-run-2 to Experiment: experiment-test-course-1


INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-test-course-1-run-2 to Experiment: experiment-test-course-1


#### Comparing runs in the same experiment

Finally, use the experiment name as a parameter to the method `get_experiment_df()` to get the results of all the runs within the experiment as a pandas dataframe.

In [54]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)

                                            0                         1
experiment_name      experiment-test-course-1  experiment-test-course-1
run_name                                run-2                     run-1
run_type                 system.ExperimentRun      system.ExperimentRun
state                                COMPLETE                  COMPLETE
param.batch_size                         32.0                      32.0
param.epochs                            100.0                     100.0
param.units                             256.0                     128.0
param.learning_rate                      0.01                      0.01
metric.test_acc                          98.8                      98.7
metric.train_acc                         99.5                      99.3


#### Delete the experiment

Next, delete the experiment using the `delete()` method.

In [55]:
exp = aiplatform.Experiment(EXPERIMENT_NAME)
try:
    exp.delete()
except Exception as e:
    print(e)

Experiment run run-2 skipped backing tensorboard run deletion.
To delete backing tensorboard run, execute the following:
tensorboard_run_artifact = aiplatform.metadata.artifact.Artifact(artifact_name=f"experiment-test-course-1-run-2-tb-run")
tensorboard_run_resource = aiplatform.TensorboardRun(tensorboard_run_artifact.metadata["resourceName"])
tensorboard_run_resource.delete()
tensorboard_run_artifact.delete()


To delete backing tensorboard run, execute the following:
tensorboard_run_artifact = aiplatform.metadata.artifact.Artifact(artifact_name=f"experiment-test-course-1-run-2-tb-run")
tensorboard_run_resource = aiplatform.TensorboardRun(tensorboard_run_artifact.metadata["resourceName"])
tensorboard_run_resource.delete()
tensorboard_run_artifact.delete()


Deleting Context : projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-test-course-1-run-2


INFO:google.cloud.aiplatform.base:Deleting Context : projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-test-course-1-run-2


Context deleted. . Resource name: projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-test-course-1-run-2


...

INFO:google.cloud.aiplatform.base:Deleting Context resource: projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-test-course-1


Delete Context backing LRO: projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-test-course-1/operations/5446936520457977856


INFO:google.cloud.aiplatform.base:Delete Context backing LRO: projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-test-course-1/operations/5446936520457977856


Context resource projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-test-course-1 deleted.


INFO:google.cloud.aiplatform.base:Context resource projects/837843744498/locations/us-central1/metadataStores/default/contexts/experiment-test-course-1 deleted.


### Create artifact lineage in experiment runs

In this example, you add artifact lineage to your experiment run. First, create an experiment and then start a run within the experiment.

In [26]:
# Create experiment
your_name = ...
EXPERIMENT_NAME = f"supaero-vertex-course-1-3-{your_name}"
aiplatform.init(experiment=EXPERIMENT_NAME)
aiplatform.start_run("run-1")

Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/example-81009a18-e9fa-11ef-8f5e-42010a800004-run-1 to Experiment: example-81009a18-e9fa-11ef-8f5e-42010a800004


INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/837843744498/locations/us-central1/metadataStores/default/contexts/example-81009a18-e9fa-11ef-8f5e-42010a800004-run-1 to Experiment: example-81009a18-e9fa-11ef-8f5e-42010a800004


#### Create a dataset and model artifacts

Next, you create synthetic artifacts in the Vertex ML Metadata to be associated with this run in the experiment, as lineage. You'll create:

- `dataset_artifact`: A dataset that is the input to the experiment run.
- `model_artifact`: A model that is the output from the experiment run.

In [27]:
DATASET_URI = "gs://example/dataset.csv"
MODEL_URI = "gs://example/saved_model.pb"

dataset_artifact = aiplatform.Artifact.create(
    schema_title="system.Dataset", display_name="example_dataset", uri=DATASET_URI
)

model_artifact = aiplatform.Artifact.create(
    schema_title="system.Model", display_name="example_modl", uri=MODEL_URI
)

#### Create the artifact lineage

Next, to create artifact lineage for an experiment run, instantiate an execution using the method `start_execution()`. You then attach input artifacts using the method `assign_input_artifacts()` and attach output artifacts using the method `assign_output_artifacts()`.

In this example, to find the lineage for the experiment, you add a synthetic (metadata) entry `lineage` to the execution run, and set the value to the console uri for the lineage, which you can get from the method `get_output_artifacts()` and the property `lineage_console_uri`.

In [28]:
with aiplatform.start_execution(
    schema_title="system.ContainerExecution", display_name="example_training"
) as execution:
    execution.assign_input_artifacts([dataset_artifact])

    # Log the params you expect
    ...
    
    # Log the metrics you expect
    ...

    execution.assign_output_artifacts([model_artifact])

    aiplatform.log_metrics(
        {"lineage": execution.get_output_artifacts()[0].lineage_console_uri}
    )

#### Get the experiment results

Next, use the experiment name as a parameter to the method `get_experiment_df()` to get the results of the experiment as a pandas dataframe.

In this example, you stored the resource URI to the lineage as a metric value `lineage` in the execution run.

In [29]:
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)

                                                                 0
experiment_name       example-81009a18-e9fa-11ef-8f5e-42010a800004
run_name                                                     run-1
run_type                                      system.ExperimentRun
state                                                     COMPLETE
param.units                                                  256.0
metric.acc                                                    96.8
metric.lineage   https://console.cloud.google.com/vertex-ai/loc...


#### Visualize the artifact lineage

Next, open the link below to visualize the artifact lineage.

In [30]:
print(
    "Open the following link:", execution.get_output_artifacts()[0].lineage_console_uri
)

Open the following link: https://console.cloud.google.com/vertex-ai/locations/us-central1/metadata-stores/default/artifacts/e1ceca8d-d8f3-4abc-ab72-d98beda9b32d?project=projet-ia-448520


#### Delete the artifact lineage

Next, use the `delete()` method to delete the artifact lineage.

In [31]:
try:
    dataset_artifact.delete()
except Exception as e:
    print(e)
try:
    model_artifact.delete()
except Exception as e:
    print(e)

Deleting Artifact : projects/837843744498/locations/us-central1/metadataStores/default/artifacts/edffb5d5-5bcf-44ba-90fe-6df228d68933


INFO:google.cloud.aiplatform.base:Deleting Artifact : projects/837843744498/locations/us-central1/metadataStores/default/artifacts/edffb5d5-5bcf-44ba-90fe-6df228d68933


Artifact deleted. . Resource name: projects/837843744498/locations/us-central1/metadataStores/default/artifacts/edffb5d5-5bcf-44ba-90fe-6df228d68933


INFO:google.cloud.aiplatform.base:Artifact deleted. . Resource name: projects/837843744498/locations/us-central1/metadataStores/default/artifacts/edffb5d5-5bcf-44ba-90fe-6df228d68933


Deleting Artifact resource: projects/837843744498/locations/us-central1/metadataStores/default/artifacts/edffb5d5-5bcf-44ba-90fe-6df228d68933


...

INFO:google.cloud.aiplatform.base:Deleting Artifact resource: projects/837843744498/locations/us-central1/metadataStores/default/artifacts/e1ceca8d-d8f3-4abc-ab72-d98beda9b32d


Delete Artifact backing LRO: projects/837843744498/locations/us-central1/metadataStores/default/artifacts/e1ceca8d-d8f3-4abc-ab72-d98beda9b32d/operations/1964822777709985792


INFO:google.cloud.aiplatform.base:Delete Artifact backing LRO: projects/837843744498/locations/us-central1/metadataStores/default/artifacts/e1ceca8d-d8f3-4abc-ab72-d98beda9b32d/operations/1964822777709985792


Artifact resource projects/837843744498/locations/us-central1/metadataStores/default/artifacts/e1ceca8d-d8f3-4abc-ab72-d98beda9b32d deleted.


INFO:google.cloud.aiplatform.base:Artifact resource projects/837843744498/locations/us-central1/metadataStores/default/artifacts/e1ceca8d-d8f3-4abc-ab72-d98beda9b32d deleted.


#### Delete the experiment

Next, you delete the experiment using the `delete()` method.

In [32]:
try:
    exp.delete()
except Exception as e:
    print(e)

Deleting Context : projects/837843744498/locations/us-central1/metadataStores/default/contexts/example-4a191ca0-e9fa-11ef-8f5e-42010a800004


INFO:google.cloud.aiplatform.base:Deleting Context : projects/837843744498/locations/us-central1/metadataStores/default/contexts/example-4a191ca0-e9fa-11ef-8f5e-42010a800004


Experiment example-4a191ca0-e9fa-11ef-8f5e-42010a800004 metadata node not found. Skipping deletion.
